In [1]:
import sys
sys.path.append('..')
from polynomial_program import PolynomialProgram

In [2]:
import functools
from collections import OrderedDict

# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes

In [3]:
M = [2, 6, 3, 6, 3]
K = [1, 4, 2, 8, 2]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]
 [16.  8. 32.]
 [ 8.  4. 16.]]


In [4]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "0000000100000"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        key = key[::-1]
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    return solution_vector_correct(key) and execution_time(key) == d


correct_machines = ['000', '001', '010', '011', '100']
machine_to_index = {'000': 0, '001': 1, '010': 2, '011': 3, '100': 4}


def solution_vector_correct(vector):
    task1_machine = vector[0:3]
    task2_machine = vector[3:6]
    task3_machine = vector[6:9]

    return task1_machine in correct_machines \
        and task2_machine in correct_machines \
        and task3_machine in correct_machines


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:3])
    task2_machine = machine_to_index.get(k[3:6])
    task3_machine = machine_to_index.get(k[6:9])


    return (0 if k[0:3] in correct_machines else 1) \
         + (0 if k[3:6] in correct_machines else 1) \
         + (0 if k[6:9] in correct_machines else 1)

In [5]:
def get_cost_model(x):
    return sum([
            cost_matrix[0, i] * (1 - x[3 * i]) * (1 - x[3 * i + 1]) * (1 - x[3 * i + 2])
            + cost_matrix[1, i] * (1 - x[3 * i]) * (1 - x[3 * i + 1]) * x[3 * i + 2]
            + cost_matrix[2, i] * (1 - x[3 * i]) * x[3 * i + 1] * (1 - x[3 * i + 2])
            + cost_matrix[3, i] * (1 - x[3 * i]) * x[3 * i + 1] * x[3 * i + 2]
            + cost_matrix[4, i] * x[3 * i] * (1 - x[3 * i + 1]) * (1 - x[3 * i + 2])
            for i in range(0, 3)
    ])


def get_machine_usage_model(x):
    return sum([
        x[3 * i] * x[3 * i + 1] * x[3 * i + 2] 
        + x[3 * i] * x[3 * i + 1] * (1 - x[3 * i + 2])
        + x[3 * i] * (1 - x[3 * i + 1]) * x[3 * i + 2]
        for i in range(0, 3)
    ])


def get_deadline_model(x):
    time_sum = sum([
            time_matrix[0, i] * (1 - x[3 * i]) * (1 - x[3 * i + 1]) * (1 - x[3 * i + 2])
            + time_matrix[1, i] * (1 - x[3 * i]) * (1 - x[3 * i + 1]) * x[3 * i + 2]
            + time_matrix[2, i] * (1 - x[3 * i]) * x[3 * i + 1] * (1 - x[3 * i + 2])
            + time_matrix[3, i] * (1 - x[3 * i]) * x[3 * i + 1] * x[3 * i + 2]
            + time_matrix[4, i] * x[3 * i] * (1 - x[3 * i + 1]) * (1 - x[3 * i + 2])
                for i in range(0, 3)
    ])
    slack_sum = 8 * x[6] + 4 * x[7] + 2 * x[8] + x[9]
    time_constraint = (d - time_sum - slack_sum) ** 2
    
    return time_constraint

In [6]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 1024
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

In [7]:
A = 1
B = 35
C = 25

In [8]:
pp = PolynomialProgram(10)

pp.add_objective(get_cost_model(pp.x), A)
pp.add_objective(get_machine_usage_model(pp.x), B)
pp.add_objective(get_deadline_model(pp.x), C)

In [9]:
qubit_op, offset = pp.to_ising()

In [10]:
compute_eigenvalues(qubit_op)

state		time	cost	machine use	correct	eigenvalue
0000010000	19.0	22.0	0		True	(-366.0156249999998+0j)
1000000000	19.0	23.0	0		True	(-365.01562499999966+0j)
0100000000	19.0	23.0	0		True	(-365.01562499999943+0j)
1000100001	19.0	24.0	0		True	(-364.01562499999966+0j)
1001000001	19.0	24.0	0		True	(-364.01562499999966+0j)
0101000001	19.0	24.0	0		True	(-364.01562499999955+0j)
0100100001	19.0	24.0	0		True	(-364.01562499999955+0j)
0000110000	19.0	26.0	0		True	(-362.0156249999996+0j)
0000010100	19.0	26.0	0		True	(-362.01562499999955+0j)
0100000100	19.0	27.0	0		True	(-361.01562499999943+0j)
1000000100	19.0	27.0	0		True	(-361.01562499999943+0j)
1000100101	19.0	28.0	0		True	(-360.0156249999998+0j)
1001000101	19.0	28.0	0		True	(-360.01562499999955+0j)
0100100101	19.0	28.0	0		True	(-360.01562499999955+0j)
0101000101	19.0	28.0	0		True	(-360.01562499999943+0j)
0010011000	19.0	28.0	0		True	(-360.0156249999992+0j)
0000110100	19.0	30.0	0		True	(-358.0156249999998+0j)
0010111000	19.0	32.0	0		True	(-356.0156

0001111110	20.0	6.0	2		False	(-287.0156249999998+0j)
0001101110	20.0	6.0	2		False	(-287.0156249999998+0j)
0101011110	18.0	8.0	2		False	(-285.0156249999999+0j)
0101101110	18.0	8.0	2		False	(-285.0156249999998+0j)
1001101110	18.0	8.0	2		False	(-285.0156249999997+0j)
1001011110	18.0	8.0	2		False	(-285.01562499999966+0j)
1001111110	18.0	8.0	2		False	(-285.01562499999966+0j)
0101111110	18.0	8.0	2		False	(-285.01562499999966+0j)
0000000000	21.0	21.0	0		False	(-267.01562499999966+0j)
0001000001	21.0	22.0	0		False	(-266.0156249999998+0j)
0000100001	21.0	22.0	0		False	(-266.0156249999998+0j)
0010000000	17.0	23.0	0		False	(-265.0156249999996+0j)
1000010000	17.0	24.0	0		False	(-264.0156249999998+0j)
0010100001	17.0	24.0	0		False	(-264.01562499999966+0j)
0011000001	17.0	24.0	0		False	(-264.01562499999966+0j)
0100010000	17.0	24.0	0		False	(-264.01562499999966+0j)
0000000100	21.0	25.0	0		False	(-263.01562499999955+0j)
0001000101	21.0	26.0	0		False	(-262.01562499999966+0j)
0000100101	21.0	26.0	0		Fal

0101001000	22.0	28.0	0		False	(-135.0156249999992+0j)
0100011001	22.0	28.0	0		False	(-135.0156249999992+0j)
0100101000	22.0	28.0	0		False	(-135.0156249999992+0j)
0110100000	16.0	32.0	0		False	(-131.0156250000001+0j)
0110010001	16.0	32.0	0		False	(-131.0156249999999+0j)
0010110101	16.0	32.0	0		False	(-131.01562499999986+0j)
0111000000	16.0	32.0	0		False	(-131.0156249999998+0j)
0100111001	22.0	32.0	0		False	(-131.01562499999932+0j)
1000111001	22.0	32.0	0		False	(-131.01562499999926+0j)
0110001001	22.0	35.0	0		False	(-128.0156249999992+0j)
0110110001	16.0	36.0	0		False	(-127.01562500000006+0j)
0110100100	16.0	36.0	0		False	(-127.015625+0j)
0111000100	16.0	36.0	0		False	(-127.01562499999989+0j)
0110010101	16.0	36.0	0		False	(-127.01562499999989+0j)
0110110101	16.0	40.0	0		False	(-123.01562499999983+0j)
0010000111	16.0	43.0	0		False	(-120.01562499999983+0j)
0000001101	22.0	9.0	1		False	(-119.0156250000004+0j)
0100010111	16.0	44.0	0		False	(-119.01562499999972+0j)
0100100110	16.0	44.0	0		Fal

0001011001	23.0	22.0	1		False	(68.98437500000018+0j)
0001101001	23.0	22.0	1		False	(68.98437500000027+0j)
0001111001	23.0	22.0	1		False	(68.9843750000006+0j)
0110111100	15.0	24.0	1		False	(70.98437499999922+0j)
0011010101	15.0	24.0	1		False	(70.98437499999997+0j)
0011100101	15.0	24.0	1		False	(70.98437500000003+0j)
0011110101	15.0	24.0	1		False	(70.98437500000027+0j)
0111100001	15.0	28.0	1		False	(74.98437499999983+0j)
0111010001	15.0	28.0	1		False	(74.98437499999991+0j)
0111110001	15.0	28.0	1		False	(74.98437500000014+0j)
0111010101	15.0	32.0	1		False	(78.98437500000003+0j)
0111100101	15.0	32.0	1		False	(78.98437500000003+0j)
0111110101	15.0	32.0	1		False	(78.98437500000017+0j)
1100001100	15.0	3.0	2		False	(84.9843749999987+0j)
1110001100	15.0	3.0	2		False	(84.98437499999875+0j)
1010001100	15.0	3.0	2		False	(84.98437499999883+0j)
1100101101	15.0	4.0	2		False	(85.98437499999864+0j)
1110101101	15.0	4.0	2		False	(85.9843749999988+0j)
1010101101	15.0	4.0	2		False	(85.98437499999885+0j)
11

1110110000	13.0	20.0	1		False	(566.9843749999997+0j)
1010010100	13.0	20.0	1		False	(566.9843749999997+0j)
1110010100	13.0	20.0	1		False	(566.9843749999997+0j)
0110011010	13.0	20.0	1		False	(566.9843749999998+0j)
1010110000	13.0	20.0	1		False	(566.9843749999998+0j)
1100010100	13.0	20.0	1		False	(566.9843749999998+0j)
0110110110	13.0	56.0	0		False	(567.9843750000003+0j)
0001110011	13.0	22.0	1		False	(568.9843749999999+0j)
0001100011	13.0	22.0	1		False	(568.9843749999999+0j)
0001010011	13.0	22.0	1		False	(568.9843749999999+0j)
1110110100	13.0	24.0	1		False	(570.9843749999994+0j)
1010110100	13.0	24.0	1		False	(570.9843749999997+0j)
1100110100	13.0	24.0	1		False	(570.9843749999998+0j)
0110111010	13.0	24.0	1		False	(570.9843749999999+0j)
1100000110	13.0	35.0	1		False	(581.9843749999995+0j)
1110000110	13.0	35.0	1		False	(581.9843749999997+0j)
1010000110	13.0	35.0	1		False	(581.9843749999997+0j)
1010100111	13.0	36.0	1		False	(582.9843749999994+0j)
1110100111	13.0	36.0	1		False	(582.98437499999

0101110011	11.0	24.0	1		False	(1270.9843750000002+0j)
1110010110	11.0	36.0	1		False	(1282.9843749999995+0j)
1100010110	11.0	36.0	1		False	(1282.9843749999998+0j)
1010010110	11.0	36.0	1		False	(1282.984375+0j)
1110011010	11.0	4.0	2		False	(1285.984374999999+0j)
1100011010	11.0	4.0	2		False	(1285.9843749999993+0j)
1010011010	11.0	4.0	2		False	(1285.9843749999993+0j)
1110110110	11.0	40.0	1		False	(1286.9843749999995+0j)
1010110110	11.0	40.0	1		False	(1286.984375+0j)
1100110110	11.0	40.0	1		False	(1286.984375+0j)
1110111010	11.0	8.0	2		False	(1289.984374999999+0j)
1010111010	11.0	8.0	2		False	(1289.9843749999995+0j)
1100111010	11.0	8.0	2		False	(1289.9843749999995+0j)
1111010111	11.0	32.0	2		False	(1313.9843749999995+0j)
1111110111	11.0	32.0	2		False	(1313.9843749999995+0j)
1111100111	11.0	32.0	2		False	(1313.9843749999995+0j)
1011100111	11.0	32.0	2		False	(1313.9843749999998+0j)
1011010111	11.0	32.0	2		False	(1313.9843749999998+0j)
1101010111	11.0	32.0	2		False	(1313.9843749999998+0j)
110

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [ ]:
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubit_op.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubit_op, ry, spsa)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result_vqe = vqe.run(quantum_instance)   
state_vector_vqe = result_vqe['eigenstate']
get_stats_for_result(state_vector_vqe)

In [ ]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)